In [2]:
# Notebook for testing YOLO speed
!pip install ultralytics

In [2]:
import cv2
import random
import time
from ultralytics import YOLO

DEFAULT_COLORS = [
    (255, 0, 0),  # Red
    (0, 255, 0),  # Green
    (0, 0, 255),  # Blue
    (255, 255, 0),  # Yellow
    (128, 0, 128)  # Purple
]


def process_frame(frame, model):
    """Process a single frame and return detection results
    :param frame: one frame to predict
    :return: result of prediction
    """
    results = model.predict(frame, verbose=False)
    return results[0]


def draw_detections(frame, detections, classes, colors):
    """Draw bounding boxes and labels on the frame
    :param frame: one frame to work with
    :param detections: list of boxes and labels witch was detected
    :return: frame with labeled objects, if they were detected
    """
    for box in detections.boxes:
        class_id = int(box.cls)
        label = classes[class_id]
        confidence = float(box.conf)

        if confidence < confidence:
            continue

        x1, y1, x2, y2 = map(int, box.xyxy[0])
        color = colors[class_id]

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        text = f"{label} {confidence:.2f}"

        (text_width, text_height), _ = cv2.getTextSize(
            text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2
        )

        text_y = y1 - 10 if y1 - 10 > text_height else y1 + 20

        cv2.rectangle(
            frame,
            (x1, text_y - text_height - 2),
            (x1 + text_width, text_y + 2),
            color,
            -1
        )

        cv2.putText(
            frame,
            text,
            (x1, text_y),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (30, 30, 30),
            2
        )
    return frame


def process_video(input_path: str,
                  output_path: str,
                  model,
                  classes,
                  colors,
                  show_live: bool = False):
    """
    Process video file and save annotated results
    :param input_path: input video file path
    :param output_path: output video file path
    :param show_live: show real-time processing window
    """
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise ValueError("Error opening video file")

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_path,
                          cv2.VideoWriter_fourcc(*'mp4v'),
                          fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = process_frame(frame, model)
        frame = draw_detections(frame, results, classes, colors)

        if show_live:
            cv2.imshow('Video Processing', frame)
            if cv2.waitKey(1) == ord('q'):
                break

        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()


def process_video_fps(input_path: str, output_path: str, model, classes, colors,
                      skip_every_n: int = None,
                      skip_probability: float = None):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise ValueError("Error opening video file")

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    frame_times = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        start_time = time.time()
        skip = False

        if skip_every_n is not None:
            if frame_count % skip_every_n == 0:
                skip = True
        elif skip_probability is not None:
            if random.random() < skip_probability:
                skip = True

        if not skip:
            results = process_frame(frame, model)
            frame = draw_detections(frame, results, classes, colors)

        out.write(frame)

        frame_time = time.time() - start_time
        frame_times.append(frame_time)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    return frame_times

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [12]:
import argparse
from pathlib import Path
import statistics
from ultralytics import YOLO
import torch


def calculate_stats(frame_times):
    if not frame_times:
        return {}

    total_frames = len(frame_times)
    print(total_frames)
    total_time = sum(frame_times)
    avg_fps = total_frames / total_time if total_time > 0 else 0

    fps_list = []
    sequence_from = []
    for i in range(total_frames):
        sequence_from.append(frame_times[i])
        if (i + 1) % 10 == 0:
            mean10 = 10.0 / sum(sequence_from)
            # print(mean10)
            fps_list.append(mean10)
            sequence_from = []
    if sequence_from:
        fps_list.append(len(sequence_from) / sum(sequence_from))

    return {
        'max_fps': max(fps_list) if fps_list else 0,
        'min_fps': min(fps_list) if fps_list else 0,
        'avg_fps': avg_fps,
        'std_fps': statistics.stdev(fps_list) if len(fps_list) > 1 else 0
    }

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def main(weights_dir, videos_dir, output_dir):
    output_dir.mkdir(exist_ok=True, parents=True)

    models = list(weights_dir.glob("*.pt"))
    videos = list(videos_dir.glob("*"))

    skip_n_params = [2, 3, 4, 5]
    skip_q_params = [0, 0.1, 0.3, 0.5]

    results = []

    for model_path in models:
        model = YOLO(model_path)
        model.to(device)
        model_name = model_path.stem

        for video_path in videos:
            video_name = video_path.stem

            for n in skip_n_params:
                output_path = output_dir / f"{model_name}_{video_name}_skip_every_{n}.mp4"
                try:
                    frame_times = process_video_fps(
                        str(video_path), str(output_path), model,
                        model.names, DEFAULT_COLORS,
                        skip_every_n=n
                    )
                    stats = calculate_stats(frame_times)
                    results.append({
                        'model': model_name,
                        'video': video_name,
                        'type': 'skip_n',
                        'param': n,
                        'stats': stats
                    })
                except Exception as e:
                    print(f"Error with {model_name} on {video_name} (n={n}): {str(e)}")

            for q in skip_q_params:
                output_path = output_dir / f"{model_name}_{video_name}_skip_prob_{q}.mp4"
                try:
                    frame_times = process_video_fps(
                        str(video_path), str(output_path), model,
                        model.names, DEFAULT_COLORS,
                        skip_probability=q
                    )
                    stats = calculate_stats(frame_times)
                    results.append({
                        'model': model_name,
                        'video': video_name,
                        'type': 'skip_q',
                        'param': q,
                        'stats': stats
                    })
                except Exception as e:
                    print(f"Error with {model_name} on {video_name} (q={q}): {str(e)}")

    for result in results:
        print(f"\nModel: {result['model']}")
        print(f"Video: {result['video']}")
        print(f"Type: {result['type']}, Param: {result['param']}")
        print(f"Max FPS: {result['stats']['max_fps']:.2f}")
        print(f"Min FPS: {result['stats']['min_fps']:.2f}")
        print(f"Avg FPS: {result['stats']['avg_fps']:.2f}")
        print(f"Std FPS: {result['stats']['std_fps']:.2f}")

In [5]:
videos_dir = Path("/kaggle/input/all-in-one/all_in_one/videos")
weights_dir = Path("/kaggle/input/all-in-one/all_in_one/weights")
output_dir = Path("/kaggle/working/results")

In [13]:
main(weights_dir, videos_dir, output_dir)

495
495
495
495
495
495
495
495

Model: fire-smoke
Video: fire-smoke
Type: skip_n, Param: 2
Max FPS: 76.63
Min FPS: 28.11
Avg FPS: 67.90
Std FPS: 6.98

Model: fire-smoke
Video: fire-smoke
Type: skip_n, Param: 3
Max FPS: 67.45
Min FPS: 49.89
Avg FPS: 59.31
Std FPS: 3.78

Model: fire-smoke
Video: fire-smoke
Type: skip_n, Param: 4
Max FPS: 60.33
Min FPS: 46.16
Avg FPS: 54.85
Std FPS: 3.62

Model: fire-smoke
Video: fire-smoke
Type: skip_n, Param: 5
Max FPS: 54.47
Min FPS: 46.42
Avg FPS: 52.21
Std FPS: 1.38

Model: fire-smoke
Video: fire-smoke
Type: skip_q, Param: 0
Max FPS: 46.32
Min FPS: 38.62
Avg FPS: 44.32
Std FPS: 1.34

Model: fire-smoke
Video: fire-smoke
Type: skip_q, Param: 0.1
Max FPS: 62.97
Min FPS: 35.97
Avg FPS: 47.80
Std FPS: 4.41

Model: fire-smoke
Video: fire-smoke
Type: skip_q, Param: 0.3
Max FPS: 103.42
Min FPS: 43.35
Avg FPS: 58.29
Std FPS: 12.56

Model: fire-smoke
Video: fire-smoke
Type: skip_q, Param: 0.5
Max FPS: 115.68
Min FPS: 43.86
Avg FPS: 73.61
Std FPS: 16.46
